In [29]:
import pandas as pd
import os
import pickle
from collections import defaultdict
from itertools import combinations
import time
from pathlib import Path
import csv
import networkx as nx

In [30]:
# Configuração do backend do matplotlib para notebooks
import os
# Remove a variável de ambiente que causa conflito
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']

import matplotlib
matplotlib.use('Agg')  # Backend não-interativo que funciona em qualquer ambiente
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Para garantir que os gráficos sejam exibidos no notebook
plt.ion()  # Ativa modo interativo

# Carregando os dados

In [14]:
root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
# garante que encontramos o dataset independentemente do cwd do notebook
input_file = root / 'datasets' / 'netflix_titles.csv'

print(f"Lendo: {input_file}")

df = pd.read_csv(input_file)

Lendo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/datasets/netflix_titles.csv


## Análise geral

In [15]:
# Etapa 2: Parse do CSV e construção das estruturas de co-atores

actor_coactors = defaultdict(set)
pair_counts = defaultdict(int)
actor_appearances = defaultdict(int)

for idx, cast in enumerate(df['cast'].fillna('')):
    if not cast:
        continue
    actors = [a.strip() for a in cast.split(',') if a.strip()]
    for a in actors:
        actor_appearances[a] += 1
    for a, b in combinations(actors, 2):
        key = tuple(sorted((a, b)))
        pair_counts[key] += 1
        actor_coactors[a].add(b)
        actor_coactors[b].add(a)

num_actors_with_coactors = len(actor_coactors)
num_actors_with_appearances = len(actor_appearances)
num_pairs = len(pair_counts)

print(f"Total de linhas no CSV: {df.shape[0]}")
print(f"Atores com pelo menos 1 co-ator: {num_actors_with_coactors}")
print(f"Atores com aparições registradas: {num_actors_with_appearances}")
print(f"Pares únicos de atores (potenciais arestas): {num_pairs}")

# Tops para inspeção
top_by_degree = sorted(((actor, len(neigh)) for actor, neigh in actor_coactors.items()), key=lambda x: x[1], reverse=True)[:20]
print('\nTop 20 por grau (número de coatores):')
for actor, deg in top_by_degree:
    print(f"{actor}: {deg}")

top_by_appearances = sorted(actor_appearances.items(), key=lambda x: x[1], reverse=True)[:20]
print('\nTop 20 por aparições:')
for actor, cnt in top_by_appearances:
    print(f"{actor}: {cnt}")

# Salvando intermediários em results/q1
out_dir = root / 'results'
out_dir.mkdir(parents=True, exist_ok=True)

# Salvar tops em CSVs para inspeção
pd.DataFrame(top_by_appearances, columns=['Actor', 'Appearances']).to_csv(out_dir / 'top_by_appearances.csv', index=False)
pd.DataFrame(top_by_degree, columns=['Actor', 'Degree']).to_csv(out_dir / 'top_by_degree.csv', index=False)

end = time.time()
print(f"\nArquivos intermediários salvos em: {out_dir}")


Total de linhas no CSV: 8807
Atores com pelo menos 1 co-ator: 36039
Atores com aparições registradas: 36439
Pares únicos de atores (potenciais arestas): 289207

Top 20 por grau (número de coatores):
Anupam Kher: 273
Samuel L. Jackson: 239
Takahiro Sakurai: 228
Fred Tatasciore: 226
Yuichi Nakamura: 223
Yuki Kaji: 220
Shah Rukh Khan: 210
Fred Armisen: 209
Akshay Kumar: 193
Katsuyuki Konishi: 191
Jun Fukuyama: 188
Om Puri: 187
Junichi Suwabe: 185
Naseeruddin Shah: 183
Boman Irani: 183
Hiroshi Kamiya: 182
James Franco: 182
Maya Rudolph: 181
Paresh Rawal: 179
Amitabh Bachchan: 178

Top 20 por aparições:
Anupam Kher: 43
Shah Rukh Khan: 35
Julie Tejwani: 33
Naseeruddin Shah: 32
Takahiro Sakurai: 32
Rupa Bhimani: 31
Akshay Kumar: 30
Om Puri: 30
Yuki Kaji: 29
Amitabh Bachchan: 28
Paresh Rawal: 28
Boman Irani: 27
Rajesh Kava: 26
Vincent Tong: 26
Andrea Libman: 25
Kareena Kapoor: 25
Samuel L. Jackson: 24
John Cleese: 24
Jigna Bhardwaj: 23
Fred Tatasciore: 23

Arquivos intermediários salvos em: /h

In [34]:
# Etapa 3 (separada): Selecionar TOP_N atores por grau (conexões)
# Não executo esta célula automaticamente — confirme quando quiser rodar.

# Parâmetro dinâmico: altere antes de executar, se desejar
TOP_N = 100

# Determina paths conforme o notebook

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir = root / 'results' / 'q1'
out_dir.mkdir(parents=True, exist_ok=True)

# Tenta usar variáveis em memória (caso a célula de parsing tenha sido executada);
# caso contrário, carrega os pickles gerados pela etapa de parsing.
try:
    actor_coactors
    pair_counts
    actor_appearances
except NameError:
    # tenta carregar arquivos em results/q1
    try:
        with open(out_dir / 'actor_coactors.pkl', 'rb') as f:
            actor_coactors = pickle.load(f)
        with open(out_dir / 'pair_counts.pkl', 'rb') as f:
            pair_counts = pickle.load(f)
        with open(out_dir / 'actor_appearances.pkl', 'rb') as f:
            actor_appearances = pickle.load(f)
        print('Carreguei pickles de', out_dir)
    except FileNotFoundError:
        raise FileNotFoundError('Estruturas de parsing não encontradas em memória nem em results/q1. Execute a célula de parsing primeiro.')

# Calcula grau (número de coatores) e seleciona TOP_N
degrees = {actor: len(neigh) for actor, neigh in actor_coactors.items()}
all_actors_count = len(degrees)
sorted_by_degree = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
top_actors = sorted_by_degree[:TOP_N]
actual_top_n = len(top_actors)
print(f"Atores disponíveis: {all_actors_count}, selecionando TOP_N = {TOP_N} => selecionados: {actual_top_n}")

top_names = [a for a, _ in top_actors]
top_set = set(top_names)

# Filtrar pares onde ambos estão no top
filtered_pairs = {pair: cnt for pair, cnt in pair_counts.items() if pair[0] in top_set and pair[1] in top_set}
num_filtered_pairs = len(filtered_pairs)
print(f"Pares no subgrafo dos TOP {actual_top_n}: {num_filtered_pairs}")

# --- Usando pandas para gerar os CSVs (mais simples e robusto) ---
# links dataframe
links_rows = [(a, b, cnt) for (a, b), cnt in filtered_pairs.items()]
links_df = pd.DataFrame(links_rows, columns=['Source', 'Target', 'Movies_Count'])
links_csv = out_dir / 'links_top.csv'
links_df.to_csv(links_csv, index=False)

# top actors dataframe
top_rows = [(a, degrees.get(a, 0), actor_appearances.get(a, 0)) for a, _ in top_actors]
top_df = pd.DataFrame(top_rows, columns=['Actor', 'Degree', 'Appearances'])
top_csv = out_dir / 'top_actors.csv'
top_df.to_csv(top_csv, index=False)

# Salvar pickles úteis
with open(out_dir / 'filtered_pair_counts.pkl', 'wb') as f:
    pickle.dump(filtered_pairs, f)
with open(out_dir / 'top_actor_names.pkl', 'wb') as f:
    pickle.dump(top_names, f)

print(f"Arquivos gerados (não para Flourish ainda):\n- {links_csv}\n- {top_csv}\n- {out_dir / 'filtered_pair_counts.pkl'}\n- {out_dir / 'top_actor_names.pkl'}")

# Resumo rápido dos 10 atores com maior grau selecionados
print('\nTop 10 (dos selecionados) por grau:')
for actor, deg in top_actors[:10]:
    print(f"{actor}: grau={deg}, aparições={actor_appearances.get(actor, 0)}")


Atores disponíveis: 36039, selecionando TOP_N = 100 => selecionados: 100
Pares no subgrafo dos TOP 100: 583
Arquivos gerados (não para Flourish ainda):
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/links_top.csv
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/top_actors.csv
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/filtered_pair_counts.pkl
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/top_actor_names.pkl

Top 10 (dos selecionados) por grau:
Anupam Kher: grau=273, aparições=43
Samuel L. Jackson: grau=239, aparições=24
Takahiro Sakurai: grau=228, aparições=32
Fred Tatasciore: grau=226, aparições=23
Yuichi Nakamura: grau=223, aparições=19
Yuki Kaji: grau=220, aparições=29
Shah Rukh Khan: grau=210, aparições=35
Fred Armisen:

In [35]:
# Etapa 4: Construir grafo NetworkX e calcular métricas (densidade, centralidades, PageRank)
# Esta célula roda a análise do grafo a partir de `filtered_pair_counts.pkl` e `top_actor_names.pkl`.

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir = root / 'results'
out_dir.mkdir(parents=True, exist_ok=True)


print(f"Nomes no top: {len(top_names)}; pares filtrados: {len(filtered_pairs)}")

# Construir grafo ponderado (undirected)
G = nx.Graph()
G.add_nodes_from(top_names)
for (a, b), w in filtered_pairs.items():
    # adicionar aresta com atributo weight
    G.add_edge(a, b, weight=w)

n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
print(f"Grafo: nós={n_nodes}, arestas={n_edges}")

# Métricas globais
density = nx.density(G)
components = list(nx.connected_components(G))
num_components = len(components)
largest_cc = max(components, key=len) if components else set()
largest_cc_size = len(largest_cc)

print(f"Densidade: {density:.6g}")
print(f"Componentes: {num_components}; maior componente: {largest_cc_size} nós")

# Centralidades
print('\nCalculando centralidades... (isso pode levar algum tempo para betweenness se exato)')
# Degree centrality (normalizada)
degree_c = nx.degree_centrality(G)
# Betweenness centrality (exata ou aproximada)
betweenness_c = nx.betweenness_centrality(G)
# Closeness centrality
closeness_c = nx.closeness_centrality(G)
# PageRank (usa weights)
pagerank = nx.pagerank(G, alpha=0.85, max_iter=100)

# Degree raw (número de vizinhos)
degree_raw = dict(G.degree())

# Montar DataFrame com métricas
metrics_df = pd.DataFrame({
    'Actor': list(G.nodes()),
    'Degree': [degree_raw.get(n, 0) for n in G.nodes()],
    'DegreeCentrality': [degree_c.get(n, 0) for n in G.nodes()],
    'Betweenness': [betweenness_c.get(n, 0) for n in G.nodes()],
    'Closeness': [closeness_c.get(n, 0) for n in G.nodes()],
    'PageRank': [pagerank.get(n, 0) for n in G.nodes()],
})

# Normalizar colunas (opcional) — apenas como colunas separadas para inspeção
metrics_df['Degree_norm'] = (metrics_df['Degree'] - metrics_df['Degree'].min()) / (metrics_df['Degree'].max() - metrics_df['Degree'].min())
metrics_df['Betweenness_norm'] = (metrics_df['Betweenness'] - metrics_df['Betweenness'].min()) / (metrics_df['Betweenness'].max() - metrics_df['Betweenness'].min())
metrics_df['Closeness_norm'] = (metrics_df['Closeness'] - metrics_df['Closeness'].min()) / (metrics_df['Closeness'].max() - metrics_df['Closeness'].min())
metrics_df['PageRank_norm'] = (metrics_df['PageRank'] - metrics_df['PageRank'].min()) / (metrics_df['PageRank'].max() - metrics_df['PageRank'].min())

# Salvar resultados
metrics_csv = out_dir / 'graph_metrics.csv'
metrics_df.to_csv(metrics_csv, index=False)
# salvar grafo em gpickle


# Mostrar top 10 por cada medida
print(f"\nMétricas salvas em: {metrics_csv}")

print('\nTop 10 por Degree:')
print(metrics_df.sort_values('Degree', ascending=False).head(10)[['Actor','Degree']])
print('\nTop 10 por Betweenness:')
print(metrics_df.sort_values('Betweenness', ascending=False).head(10)[['Actor','Betweenness']])
print('\nTop 10 por Closeness:')
print(metrics_df.sort_values('Closeness', ascending=False).head(10)[['Actor','Closeness']])
print('\nTop 10 por PageRank:')
print(metrics_df.sort_values('PageRank', ascending=False).head(10)[['Actor','PageRank']])

# Guardar as variáveis no notebook para uso futuro
G_graph = G
filtered_pairs_graph = filtered_pairs
metrics = metrics_df

print('\nEtapa 4 concluída — valide os resultados antes de prosseguirmos para as centralidades mais custosas (se necessário) ou detecção de comunidades.')


Nomes no top: 100; pares filtrados: 583
Grafo: nós=100, arestas=583
Densidade: 0.117778
Componentes: 1; maior componente: 100 nós

Calculando centralidades... (isso pode levar algum tempo para betweenness se exato)

Métricas salvas em: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/graph_metrics.csv

Top 10 por Degree:
               Actor  Degree
4    Yuichi Nakamura      27
2   Takahiro Sakurai      26
5          Yuki Kaji      25
10      Jun Fukuyama      25
25       Daisuke Ono      24
75  Nobuhiko Okamoto      23
43     Kana Hanazawa      23
42         Ai Kayano      23
50  Miyuki Sawashiro      22
12    Junichi Suwabe      22

Top 10 por Betweenness:
                 Actor  Betweenness
30       Kari Wahlgren     0.417440
0          Anupam Kher     0.173614
87  Chloë Grace Moretz     0.126796
83       Gerard Butler     0.096958
89       Paul Giamatti     0.095672
27     Elizabeth Banks     0.070178
3      Fred Tatasciore

In [36]:
# Etapa 6: Louvain + Geração de arquivos para Flourish
# Detecta comunidades com Louvain e gera links.csv e points.csv para visualização no Flourish

from networkx.algorithms.community import louvain_communities
from networkx.algorithms.community.quality import modularity
import pandas as pd
import pickle
from pathlib import Path

# Parâmetros ajustáveis
LOUVAIN_RESOLUTION = 1  # padrão 1.0; aumentar => mais comunidades
USE_LARGEST_CC = True
LOUVAIN_SEED = 42  # seed para reprodutibilidade

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir = root / 'results' / 'grafo_coautoria'
out_dir.mkdir(parents=True, exist_ok=True)

# Carregar grafo se necessário
try:
    G
    filtered_pairs
except NameError:
    raise FileNotFoundError('Grafo não encontrado. Execute a célula que constrói o grafo (Etapa 4) primeiro.')

# Subgrafo onde rodar Louvain
if USE_LARGEST_CC:
    comps = list(nx.connected_components(G))
    largest = max(comps, key=len)
    G_louvain = G.subgraph(largest).copy()
else:
    G_louvain = G

print(f"Rodando Louvain (NetworkX nativo) em n={G_louvain.number_of_nodes()} nós, m={G_louvain.number_of_edges()} arestas")
print(f"Parâmetro de resolução: {LOUVAIN_RESOLUTION}, seed: {LOUVAIN_SEED}")

# Executar Louvain (retorna lista de sets/frozensets)
communities_sets = louvain_communities(G_louvain, weight='weight', resolution=LOUVAIN_RESOLUTION, seed=LOUVAIN_SEED)

# Converter para lista de listas
communities_list = [list(c) for c in communities_sets]

# Calcular modularidade
mod = modularity(G_louvain, communities_list, weight='weight')
num_communities = len(communities_list)

print(f"\nModularidade obtida: {mod:.3f}")
print(f"Número de comunidades: {num_communities}")

# Estatísticas das comunidades
sizes = pd.Series([len(c) for c in communities_list])
print(f"\nDistribuição de tamanhos (min={sizes.min()}, max={sizes.max()}, mean={sizes.mean():.1f}, median={sizes.median():.1f})")

# Criar dicionário node -> community_id
partition = {}
for cid, comm in enumerate(communities_list):
    for node in comm:
        partition[node] = cid

# Estender partição para todos os nós do grafo original
comm_dict_louvain = {}
for node in G.nodes():
    if node in partition:
        comm_dict_louvain[node] = partition[node]
    else:
        comm_dict_louvain[node] = -1  # Nós fora do maior componente

print("\n" + "="*80)
print("GERANDO ARQUIVOS PARA FLOURISH")
print("="*80)

# ========== 1. GERAR links.csv ==========
# Filtrar apenas arestas do subgrafo Louvain
links_data = []
for (a, b), weight in filtered_pairs.items():
    # Incluir apenas se ambos estão no G_louvain
    if a in G_louvain.nodes() and b in G_louvain.nodes():
        links_data.append((a, b, weight))

links_df = pd.DataFrame(links_data, columns=['Source', 'Target', 'Value'])
links_csv = out_dir / 'links.csv'
links_df.to_csv(links_csv, index=False)

print(f"\n✅ links.csv gerado:")
print(f"   - {len(links_df)} arestas")
print(f"   - Arquivo: {links_csv}")

# ========== 2. GERAR points.csv ==========
points_data = []
for actor in G_louvain.nodes():
    points_data.append((
        actor,  # id
        comm_dict_louvain[actor]  # Community
    ))

points_df = pd.DataFrame(points_data, columns=['id', 'Community'])
points_csv = out_dir / 'points.csv'
points_df.to_csv(points_csv, index=False)

print(f"\n✅ points.csv gerado:")
print(f"   - {len(points_df)} atores")
print(f"   - {points_df['Community'].nunique()} comunidades")
print(f"   - Arquivo: {points_csv}")

# ========== 3. SALVAR METADADOS ==========
# Salvar stats de Louvain
louvain_stats = {
    'resolution': LOUVAIN_RESOLUTION,
    'seed': LOUVAIN_SEED,
    'modularity': mod,
    'num_communities': num_communities,
    'nodes_analyzed': G_louvain.number_of_nodes(),
    'edges_analyzed': G_louvain.number_of_edges()
}

pd.DataFrame([louvain_stats]).to_csv(out_dir / 'louvain_stats.csv', index=False)

# Salvar partição em pickle
with open(out_dir / 'communities_louvain.pkl', 'wb') as f:
    pickle.dump(comm_dict_louvain, f)

# ========== 4. RESUMO DAS COMUNIDADES ==========
print("\n" + "="*80)
print("RESUMO DAS COMUNIDADES")
print("="*80)

comm_sizes = points_df['Community'].value_counts().sort_values(ascending=False)
print(f"\nTop 10 maiores comunidades (ID: tamanho):")
for idx, (comm_id, size) in enumerate(comm_sizes.head(10).items(), 1):
    print(f"  {idx}. Comunidade {comm_id}: {size} atores")

print("\n" + "="*80)
print("ARQUIVOS FLOURISH PRONTOS!")
print("="*80)
print(f"\n📁 Diretório: {out_dir}")
print(f"   ├── links.csv    ({len(links_df)} arestas)")
print(f"   ├── points.csv   ({len(points_df)} nós, {num_communities} comunidades)")
print(f"   ├── louvain_stats.csv")
print(f"   └── communities_louvain.pkl")
print("\n✨ Importe links.csv e points.csv no Flourish Network Graph!")
print("   - Use 'id' como node identifier")
print("   - Use 'Community' para colorir os nós")

Rodando Louvain (NetworkX nativo) em n=100 nós, m=583 arestas
Parâmetro de resolução: 1, seed: 42

Modularidade obtida: 0.507
Número de comunidades: 3

Distribuição de tamanhos (min=12, max=61, mean=33.3, median=27.0)

GERANDO ARQUIVOS PARA FLOURISH

✅ links.csv gerado:
   - 583 arestas
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_coautoria/links.csv

✅ points.csv gerado:
   - 100 atores
   - 3 comunidades
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_coautoria/points.csv

RESUMO DAS COMUNIDADES

Top 10 maiores comunidades (ID: tamanho):
  1. Comunidade 2: 61 atores
  2. Comunidade 1: 27 atores
  3. Comunidade 0: 12 atores

ARQUIVOS FLOURISH PRONTOS!

📁 Diretório: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_coautoria
   ├── links.csv    (583 arestas)
   ├